In [65]:
import os
import glob
import pandas as pd
import numpy as np

In [66]:
### There are two ways to merge weather data with the REI reviews
#### Option A: Download weather data, put it into folders by year, and run blocks 2&3.
##### ^Only do this if you don't want to use the same time frame and data I do
#### Option B (Better!): Just use the data I posted on OSF, and skip to block 4 (this is #2)

# Create list of all file names to read in
all_filenames = []
years = ['2016',
         '2017',
         '2018',
        '2019',
        '2020',
        '2021',
        '2022']

for year in years:
    for i in glob.glob(f'{year}/*.csv'):
        all_filenames.append(i)

        # Which attributes do you want?
atts = ['STATION',
        'DATE',
        'LATITUDE',
        'LONGITUDE',
        'ELEVATION',
        'TEMP',
        'MAX',
        'MIN',
        'PRCP',
        'PRCP_ATTRIBUTES']

## Now, combine all files in the list
# Run the line below.  OR just read in the below data, and skip the next cell
d = pd.concat([pd.read_csv(f, usecols = atts) for f in all_filenames])

d = d[d['STATION'] != 99401499999]
d = d[d['STATION'] != 99999900178]
d = d[d['STATION'] != 72055399999]
d = d[d['STATION'] != 74594493784]
d = d[d['STATION'] != 99404899999]

In [72]:
### Now! Calculate averages for 3 and 7 days weather

# Write a date that is too early to calculate 7-day average
tooearly7 = 1452247200
# Write a date that is too early to calculate 3-day average
tooearly3 = 1451815200

# Make date unix so we can use the above
d['DATE_unix'] = pd.to_datetime(d.DATE).map(pd.Timestamp.timestamp)
d.reset_index(drop = True, inplace=True) #write window fn
window_size7 = 7
window_size3 = 3

i = 0
# Initialize an empty list to store moving averages
MAX_averages7 = ['NA','NA','NA','NA','NA','NA']
MIN_averages7 = ['NA','NA','NA','NA','NA','NA']
TEMP_averages7 = ['NA','NA','NA','NA','NA','NA']
PRCP_averages7 = ['NA','NA','NA','NA','NA','NA']

MAX_averages3 = ['NA','NA']
MIN_averages3 = ['NA','NA']
TEMP_averages3 = ['NA','NA']
PRCP_averages3 = ['NA','NA']
  
# Loop through the array to consider
# every window of size 7
while i < len(d.DATE) - window_size7 + 1:
    if d.DATE_unix[i] < tooearly7:
        MAX_averages7.append('NA')
        MIN_averages7.append('NA')
        TEMP_averages7.append('NA')
        PRCP_averages7.append('NA')
        i += 1
    else:
        MAX_averages7.append(round(sum(d.MAX[i : i + window_size7]) / window_size7, 4))
        MIN_averages7.append(round(sum(d.MIN[i : i + window_size7]) / window_size7, 4))
        TEMP_averages7.append(round(sum(d.TEMP[i : i + window_size7]) / window_size7, 4))
        PRCP_averages7.append(round(sum(d.PRCP[i : i + window_size7]) / window_size7, 4))
        i += 1

d['avg7MAX'] = MAX_averages7
d['avg7MIN'] = MIN_averages7
d['avg7TEMP'] = TEMP_averages7
d['avg7PRCP'] = PRCP_averages7

# Loop through the array to consider
# every window of size 3
i=0
while i < len(d.DATE) - window_size3 + 1:
    if d.DATE_unix[i] < tooearly3:
        MAX_averages3.append('NA')
        MIN_averages3.append('NA')
        TEMP_averages3.append('NA')
        PRCP_averages3.append('NA')
        i += 1
    else:
        MAX_averages3.append(round(sum(d.MAX[i : i + window_size3]) / window_size3, 2))
        MIN_averages3.append(round(sum(d.MIN[i : i + window_size3]) / window_size3, 2))
        TEMP_averages3.append(round(sum(d.TEMP[i : i + window_size3]) / window_size3, 2))
        PRCP_averages3.append(round(sum(d.PRCP[i : i + window_size3]) / window_size3, 2))
        i += 1

d['avg3MAX'] = MAX_averages3
d['avg3MIN'] = MIN_averages3
d['avg3TEMP'] = TEMP_averages3
d['avg3PRCP'] = PRCP_averages3
d = d.drop(columns=['DATE_unix'])

# If you wrote a new NOAA file, run this line to save it
d.to_csv( "NOAA_160101_221001.csv", index=False, encoding='utf-8-sig')

In [80]:
## If you downloaded the weather data from OSF, skip to here and uncomment these lines
# Weather data
d = pd.read_csv("NOAA_160101_221001.csv") 
# Station data
stations = d[['STATION','LATITUDE','LONGITUDE']].drop_duplicates()
# Reviews data (for finding stations closest to a review)
df = pd.read_csv("REI_reviews_located_clean.csv", usecols = ['lat_mid', 'lng_mid'], encoding = 'unicode_escape')
df = df[df['lat_mid'].notnull()].drop_duplicates()

# Now, write the function that finds the closest station to each review
stations = stations.set_index('STATION')
def dist(lat1, long1, lat2, long2):
    return ((lat1*110-lat2*110)**2+(long1*84-long2*84)**2)**(1/2)

def find_station(lat, long):
    stations['distance'] = stations.apply(
        lambda row: dist(lat, long, row['LATITUDE'], row['LONGITUDE']), 
        axis=1)
    return stations['distance'].idxmin()

# Find those stations for each unique lat/long from the reviews
df['STATION'] = df.apply(
    lambda row: find_station(row['lat_mid'], row['lng_mid']), 
    axis=1)

# Now, read in the reviews. Read in ALL data this time (df was only reading in lat/long)
revs = pd.read_csv("REI_reviews_located_clean.csv", encoding = 'unicode_escape')

# For speed, only include reviews with locations
revs = revs[revs['lat_mid'].notnull()]

# Merge reviews with weather stations closest to each
revs = revs.merge(df, how = 'left', on=['lat_mid', 'lng_mid'])

# Format date in the weather data so we can merge those two
d['date'] = d['DATE'].str.slice(stop=10)
# Format date in the reviews data so we can merge those two
revs['date'] = revs['SubmissionTime'].str.slice(stop=10)

# Merge and write the reviews data
revs = revs.merge(d, how = 'left', on=['STATION', 'date'])

revs.to_csv('REI_located_weathered_reviews.csv', index=False, encoding='utf-8-sig')